In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
#from data_twitter import *
#from gnn_twitter import GNN_Twitter
import random
import json
import time
import math
import sys
#from tqdm import tqdm
import statistics
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import glob, os

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

from gensim.models import FastText

import operator
import json

from bert_embedding import BertEmbedding
import mxnet as mx
import tqdm


# Finish constructing the dataset

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

def load(file, direct = 'pre/opinosis/'):
    arr = []
    with open(direct + file) as opin:
        for line in opin:
            arr.append(line)
    return(arr)
def dirload(file, direct = 'pre/opinosis/'):
    arr = ""
    with open(direct + file) as opin:
        for line in opin:
            arr += line
    if arr == "":
        return None
    else:
        return arr

def fixdoubleindent(arr):      
    narr = np.array(arr)
    narr = narr[np.where(narr != '\n')]
    return narr


In [3]:
def aggressive_fix_punc(string):
    string = string.replace('\n','')
    string = string.replace('.', '')
    string = string.replace('!', '')
    string = string.replace('?', '')
    string = string.replace(',', '')
    return(string)    
def fix_punc(string):
    string = string.replace('\n','')
    string = string.replace(' . ', '.')
    string = string.replace(' !', '!')
    string = string.replace(' ?', '?')
    string = string.replace(' ,', ',')

    return(string)
def fix_punc2(string):
    string = string.replace('\n','')
    string = string.replace(' .', '.')
    string = string.replace(' !', '!')
    string = string.replace(' ?', '?')
    string = string.replace(' ,', ',')
    return(string)
def narr_fix_punc(narr):
    for i in range(len(narr)):
        narr[i] = fix_punc(narr[i])
    return(narr)
def anarr_fix_punc(narr):
    for i in range(len(narr)):
        narr[i] = aggressive_fix_punc(narr[i])
    return(narr)
def fix_jumps(narr):
    toret = []
    for string in narr:

        string = string.split('.')
        #if isinstance(string, list):
        #    string = flatten(string)
        toret.append(string)


    return flatten(toret)

# Begin constructing the edges for our graph

In [4]:

def computesim(nparr):
    vect = TfidfVectorizer(min_df=1)
    tfidf = vect.fit_transform(nparr)

    sim_matrix = (tfidf * tfidf.T).A
    return(sim_matrix)

In [5]:
#print(sim_matrix)

In [6]:
#Exclude everything along the diagonal (and duplicates)
def simadj(sim_mat, thres = 1.5):
    nlen = sim_mat.shape[0]
    df = sim_mat[np.where(sim_mat != 1)]

    sd = statistics.stdev(df.flatten())
    mu = statistics.mean(df.flatten())

    out_matrx = np.zeros((nlen, nlen))
    #Could not get np.where working for this, so I did it manually
    #The threshold for determining if an element is an outlier
    for i in range(nlen):
        for j in range(nlen):
            if i != j:
                if sim_mat[i,j] > mu + thres * sd:
                    out_matrx[i,j] = 1
            #No need for <, as we only care about points on the other end of the extreme spectrum
    return(out_matrx)
#out_matrx is our adj matrix. I specifically avoided connecting the last sentence of a document
#to the first sentence of all others on this data set, since each documnt is only once sentence

#Actually I take the latter half of the above back, I figured out how the dataset is formatting different
#sentences of the same article. If the sentence does not begin with a space, then then it is from
#the same article
#print(np.where(out_matrx== 1))

In [7]:
def advanced_punc_fix(string):
    return string 
    '''
    puncs = ['.','?','!']
    
    #Easiest case first
    if string[-1] not in puncs:
        string = string +' . '
    out = ''
    #Tokenize the text. If the first letter of a word is capital, insert a period before it
    for s in string.split():
        if s[0].isupper():
            out = out + '. ' + s + ' '
        else:
            out = out + s + ' '
    string = out[1:len(out)]
    return(string)
    '''
def lasttofirst_str(nparr):
    paragraphs = []
    string = ''
    for i in range(len(nparr)):
        string = string + nparr[i]
        if not nparr[i].startswith(' '):
            strs = advanced_punc_fix(string).split()
            for j in range(len(strs)):
                strs[j] = strs[j].split('\'')
            strs = flatten(strs)
            for j in range(len(strs)):
                strs[j] = strs[j].split('-')
            strs = flatten(strs)
            paragraphs.append(strs)
            string = ''
    return paragraphs
def lasttofirst_indx(nparr):
    paragraphs = [list()]
    j = 0
    for i in range(len(nparr)):
        if not nparr[i].startswith(' '):
            try:
                paragraphs[j].append(i)
            except:
                paragraphs.append(list())
                print(len(paragraphs))
                paragraphs[j].append(i)
        else:
            j += 1
    return paragraphs

def lasttofirst(indx, out,nparr):
    
    paragraphs = indx
    #Make sure that the last sentence of our document points to 
    #the first sentence of every other document except
    #itself
    for i in paragraphs:
        for j in paragraphs:
            if i != j:
                #j is the first sentence and the last sentence 
                if(j - 1 in paragraphs or j == 0 or (j == len(nparr) - 1)):
                    out[i,j] = 1
                else:
                    out[i,j+1] = 1
    return(out)

def load_from_JSON(direct):
    full_stops = ['.', ' . ', '. ', ' .',
                  '?', ' ? ', '? ', ' ?',
                  '!', ' ! ', '! ', ' !']
    articles = []
    words = list()
    words.append(list())
    paragraphs = list()
    with open(direct) as json_file:
        data = json.load(json_file)
        obj = data['articles']
        sent_i = 0
        for i in range(len(obj)):
            articles.append([])
            for word in obj[i]:
                articles[i].append(word)
                if "." == word or ".\n" == word:
                    sent_i += 1
        #Compile everything into a set of sentences
        sent_count = sent_i
        num_articles = len(obj)
        sent_num = 0
        word_num = 0
        
        for i in range(num_articles):
            for j in range(len(articles[i])):
                try:
                    words[sent_num].append(articles[i][j])                    
                except:
                    words.append(list())
                    words[sent_num].append(articles[i][j])
                word_num += 1
                #Next sentence
                if articles[i][j] in full_stops:
                    sent_num += 1
                    word_num = 0
                if sent_num >= sent_count:
                    break
            if sent_num >= sent_count:
                sent_num = 0
                break
            sent_num += 1
            word_num = 0
            words.append(list())
            paragraphs.append(sent_num)
    return words, paragraphs
        
        
        


# Now we can combine everything into a single function

In [12]:
from tqdm import tnrange, tqdm_notebook


from pathlib import Path
def computeadjmatrices(path):
    adjmats = []
    files = os.listdir(path)
    max_size = 0
   
    for file in tqdm_notebook(files):
        try:
            mFile = Path("../sum_data/opinosis_graphs/" + file.replace('.txt','') + ".csv")
            #if mFile.is_file():
            #    continue
            words, paragraphs = load_from_JSON(path + file)
            forprocessing = list(map(lambda x: " ".join(x), words))

            out = simadj(computesim(forprocessing), thres = 1.2)

            out = lasttofirst(paragraphs, out, forprocessing)
            size = out.shape[0]
            out = out + np.eye(size,size,1)
            #This is where we can make sure no edge has been counted for multiple times, or any
            #other error if I think of it later
            for i in range(size):
                for j in range(size):
                    #We made an oopsie
                    if out[i,j] > 1:
                        out[i,j] = 1

            adjmats.append(out)
            if file == "0f9e4dd3454f98f77bfa02cc37dd2f6de08f3d69.txt":
                print("blep")
            np.savetxt("../sum_data/opinosis_graphs/" + file.replace('.txt','') + ".csv", out, delimiter=",")
        except:
            continue
        
        #max_size = max(max_size, len(nfile))
        
    return(adjmats)
%%capture output
computeadjmatrices("../sum_data/final/")


SyntaxError: invalid syntax (<ipython-input-12-6d9dc5e7feda>, line 41)

In [9]:
print("blep")

blep


So in this dataset, the maximum adj size is 575. We can construct the GNN with this in mind.

# We need to determine our vocabulary size

In [12]:
import nltk
nltk.download('stopwords')
  
set(stopwords.words('english'))
#Go through all of the files and determine the vocabulary
def computeunq(path):
    words = []
    files = os.listdir(path)
    for file in tqdm.tqdm(files):
        nfile = load(file)
        if not nfile:
            continue
        try:
            nfile = narr_fix_punc(fixdoubleindent(nfile))
        
            for elem in nfile:
                words.append(word_tokenize(elem))
        except:
            continue
            
    flat_list = [item for sublist in words for item in sublist]
    words = list(set(flat_list))
    return(words)

#unq_words = computeunq("pre/opinosis")
#print(unq_words)

#print(len(unq_words))

[nltk_data] Downloading package stopwords to /home/louis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 312136/312136 [47:51<00:00, 61.42it/s]  


769712


In [18]:
'''import multiprocessing

pool = multiprocessing.Pool()
flat_list = list(map(lambda word: word.lower(), unq_words))
unq_words = list(set(flat_list))
'''

In [25]:
#print(len(unq_words))

767963
moodboards


# Construct W2V embeddings

In [ ]:
#Loads one giant string of all of the text
def directloadtext(path):
    string = ''
    files = os.listdir(path)

    for file in files:
        nfile = load(file)
        if not nfile:
            continue
        try:
            nfile = narr_fix_punc(fixdoubleindent(nfile))
        
            for elem in nfile:
                string = string + "\n" + elem
        except:
            continue
    return string


def loadDataW2V(path):
    punc = [".", "!", "?"]
    string = []
    curstring = []
    files = os.listdir(path)

    for file in tqdm.tqdm(files):
        curstring = dirload(file,direct=path)
        if curstring == None:
            continue
        try:
            curstring = [fix_punc2(curstring)]
            #Split on all of the different punctuation marks
            for elem in punc:
                for i in range(len(curstring)):
                    curstring[i] = curstring[i].split(elem)
                curstring = flatten(curstring)
                for i in range(len(curstring)):
                    curstring[i] = curstring[i].lower()
            for i in range(len(curstring)):
                curstring[i] = curstring[i].split()
                try:
                    for j in range(len(curstring[i])):
                        curstring[i][j] = curstring[i][j].split('\'')
                except:
                    curstring[i] = curstring[i].split('\'')
        except:
            continue
        string += curstring
    return(string)
    
#text = directloadtext("pre/opinosis/")

total_text = loadDataW2V("pre/opinosis/") +  loadDataW2V("sum/opinosis/")

 31%|███       | 97503/312136 [15:16<1:00:02, 59.58it/s]

In [ ]:
model_embeddings = FastText(total_text, size=128, window=5, min_count=5,iter=20)
wv = model_embeddings.wv

In [ ]:
from gensim.test.utils import get_tmpfile

wv.save_word2vec_format("../sum_data/w2v_embeddings.kv", binary = False)

In [ ]:
#print(wv.vocab)

# Construct the *_vocab.json file

In [ ]:
def word_count(inp):
    counts = dict()
    tokens = anarr_fix_punc(flatten(inp))

    for token in tokens:
        if token in counts:ok
            counts[token] += 1
        else:
            counts[token] = 1

    return counts

def Reverse(tuples): 
    new_tup = tuples[::-1] 
    return new_tup 

count = word_count(total_text)
sorted_count = Reverse(sorted(count.items(), key=operator.itemgetter(1)))

In [ ]:
#Dump json to file
with open("../sum_data/opinosis_vocab.json", 'w') as f:
    json.dump(sorted_count, f)

# Construct the *_user.json file

In [27]:
from tqdm import tnrange, tqdm_notebook

def loadgold(path):
    gold_arr = []
    files = os.listdir(path)
    for file in files:
        data = load(file, path)
        if not data:
            continue
        string = word_tokenize(data[0])        
        gold_arr.append(string)
    return(gold_arr)

    
def consJSON_topics(path):
    files = os.listdir(path)
    i = 0
    #Which article are we gonna try to produce the summary from
    article_to_check = 0
    for file in tqdm_notebook(files):
        topic_name = file.replace('.txt', '')
        
        data = load(file, path)
        if not data:
            continue
        string = word_tokenize(data[0])      
        for j in range(len(string)):
            string[j] = string[j].split('-')
        gold_string = flatten(string)
        
        #What we will be saving
        data={}


        nfile = load(file)
        if not nfile:
            continue
        try:
            nfile = fixdoubleindent(nfile)

            paragraphs = lasttofirst_str(nfile)
            '''
            with open("PGMMR/opinosis/" + topic_name + ".txt", 'w') as f:
                for article in paragraphs:
                    for elem in range(len(article)):
                        if article[elem] in ['.', '?', '!','\'']:
                            article[elem] = article[elem] + "\n"
                    string = " ".join(article)
                    string = string.replace(" .\n", ".\n")
                    string = string.replace(" ?\n", "?\n")
                    string = string.replace(" !\n", "!\n")
                    string = string.replace(" ,", ",")
                    string = string.replace(" \'", "\'")
                    string = string.replace("\n ", "\n")
                    f.write(string + "\n")

            '''
            #print(paragraphs)
            sent_end = 0
            for j in range(len(paragraphs[article_to_check])):
                string = paragraphs[article_to_check][j]
                if string in {".","?","!",".\n","?\n","!\n"}:
                    sent_end = j
                    break

            isCnnDM = True
            try:
                #Is the name in hexidecimal
                int(topic_name, 16)
            except:
                isCnnDM = False
            data['id'] = i
            data['name'] = topic_name
            if not isCnnDM:
                data['gold'] = gold_string
                data['articles'] = paragraphs
            else:
                data['gold'] = gold_string
                data['articles'] = [flatten(paragraphs)]
            with open("../sum_data/final/"+ topic_name + ".json", 'w') as f:
                json.dump(data,f)


        except:
            i += 1
            continue
        i+=1
consJSON_topics('sum/opinosis/')


In [21]:
int("1eafcb17de379548083ff29304ddfa97e5836f7b", 16)

175190045450373850465899563700925111600060526459

 27%|██▋       | 85577/312136 [02:20<05:19, 708.10it/s]

# Generate Embeddings

In [ ]:
#This code is very poorly optimized! Be warned



def constructBertEmbeddings(path, ftype = "pre"):
    punc = [".", "!", "?"]
    ctx = mx.gpu(0)
    bert = BertEmbedding(ctx=ctx)    
    string = []
    files = os.listdir(path)

    for file in files:
        string = dirload(file,direct=path)
        if string == None:
            continue
        string = [fix_punc2(dirload(file,direct=path))]
        #Split on all of the different punctuation marks
        for elem in punc:
            for i in range(len(string)):
                string[i] = string[i].split(elem)
            string = flatten(string)
        result = bert(string)
        npresult = np.array(result)
        with open("../sum_data/bert_embeddings/" + ftype + "/" + file,"wb") as file:
            pickle.dump(npresult, file)
            
#constructBertEmbeddings("pre/opinosis/")
#constructBertEmbeddings("sum/opinosis/", "sum")

In [ ]:
#print(len(npresult[3][0]))

In [ ]:
#arr = pickle.load(open("../sum_data/embeddings/sum/battery-life_amazon_kindle.txt","rb"))

In [ ]:
#print(arr[1][1])

In [ ]:
#print(total_text)